# 12. Exception Handling

## Exceptions Hierarchy
---

There are **two types of exceptions** in $.NET\,\,Framework$: 
1. exceptions thrown by the *applications* we develop (`ApplicationException`)
2. exceptions thrown by the *runtime* (`SystemException`). 
   
Each of these is a **base class** within a **hierarchy of `Exception` classes**:

<img src = "_img/ExceptionHeirarchy.jpg" style="display: block; margin: auto;"/>

<br>

### The `Exception` Class

In $.NET\,\,Framework$, `Exception` is the **base class for all exceptions**.    

Several classes inherit directly from it, including `ApplicationException` and `SystemException`.    
   
These two classes are base classes for almost all exceptions that occur during the program execution.   
   
The `Exception` class contains a copy of the call-stack at the time the exception instance was created. The class also has a (usually) short message describing the error (filled in by the method throwing the exception). 
   
Every exception could have a **nested exception** also sometimes called an i**nner exception**, **wrapped exception** or **internal exception**.   
   
The ability to wrap an exception with another exception is very useful in some cases and allows exceptions to be linked in the so called **exception chain**.